# Libraries

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time
from datetime import timedelta
import datetime 
import matplotlib.pyplot as plt
import io
import urllib, base64
from flask import Flask, request

# CSS form

In [ ]:
css_style="""<style>
body {
  display: flex;
  justify-content: center;
  align-items: center;
  height: 100vh;
}
form {
  background-color: #f2f2f2;
  padding: 20px;
  border-radius: 10px;
  box-shadow: 2px 2px 5px #888888;
  text-align: center;
}
input[type="email"], input[type="password"], input[type="date"] {
  padding: 10px;
  font-size: 16px;
  margin: 10px 0;
  width: 100%;
  box-sizing: border-box;
  border-radius: 5px;
  border: 1px solid #888888;
}
input[type="submit"] {
  padding: 10px 20px;
  background-color: #4CAF50;
  color: white;
  border-radius: 5px;
  border: none;
  font-size: 16px;
  cursor: pointer;
}
.data {
  width: 50%;
  margin: 20px auto;
  border-collapse: collapse;
}
.data td, .data th {
  border: 1px solid #dddddd;
  padding: 8px;
  text-align: left;
}
.data tr:nth-child(even) {
  background-color: #dddddd;
}
</style>"""

In [ ]:
css_style_output="""<style>
body {
  font-family: Arial, sans-serif;
  font-size: 14px;
  line-height: 1.5;
  color: #333;
}

table {
  border-collapse: collapse;
  width: 100%;
}

th, td {
  border: 1px solid #ddd;
  padding: 8px;
  text-align: left;
}

th {
  background-color: #f2f2f2;
  font-weight: bold;
}

.graph-container {
  display: flex;
  align-items: center;
  justify-content: center;
  height: 400px;
}

.graph {
  background-color: #fff;
  padding: 20px;
  box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
  border-radius: 10px;
}

h1, h2, h3, h4, h5, h6 {
  margin: 0;
  padding: 0;
  font-weight: normal;
}

h1 {
  font-size: 32px;
  margin-bottom: 20px;
}

h2 {
  font-size: 24px;
  margin-bottom: 20px;
}

h3 {
  font-size: 18px;
  margin-bottom: 20px;
}
</style>"""

# Functions

In [ ]:
app = Flask(__name__)

def myfitnesspal_signin (myfitnesspal_email, myfitnesspal_password):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("https://www.myfitnesspal.com/account/login")

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "sp_message_iframe_753077")))
    driver.switch_to.frame(driver.find_element(By.ID, "sp_message_iframe_753077"))
    driver.find_element(By.XPATH,'//button[contains(text(), "AGREE AND PROCEED")]').click()
    driver.switch_to.default_content()

    time.sleep(1)
    email = driver.find_element(By.ID, "email").send_keys(myfitnesspal_email)
    password = driver.find_element(By.ID, "password").send_keys(myfitnesspal_password)
    
    time.sleep(1)
    driver.find_element(By.XPATH,'//*[@id="__next"]/div/main/div/div/form/div/div[2]/button[1]').click()

    # waiting to load the next page
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "css-m5vuyl")))

    return driver

def get_totals (driver, date):  
    driver.get("https://www.myfitnesspal.com/food/diary/?date="+date)
    time.sleep(1)
    td_elements = driver.find_elements(by=By.CSS_SELECTOR, value="tr.total td")
    values = []
    for td_element in td_elements:
        values.append(td_element.text)
    values = [value for value in values if value != ""]
    chunks = [values[i:i + 7] for i in range(0, len(values), 7)]
    df = pd.DataFrame(chunks, columns=["date", "calories", "carbs", "fat", "protein", "sodium", "sugar"])
    if len(df) != 0:
        df_totals = pd.DataFrame(df.iloc[0]).T
        df_totals.iloc[0]['date'] = date
    else:
        df_totals = df.T    
    df_totals.iloc[0]['date'] = date    
    return df_totals

def get_myfitnesspal_data(myfitnesspal_email,myfitnesspal_password,start_date,end_date):
    driver = myfitnesspal_signin (myfitnesspal_email, myfitnesspal_password)
     
    df = pd.DataFrame()
    date_format = "%Y-%m-%d"
    start_date = datetime.datetime.strptime(start_date, date_format).date()
    end_date = datetime.datetime.strptime(end_date, date_format).date()
    
    for i in range((end_date - start_date).days + 1):
        current_date = start_date + timedelta(i)
        
        df = pd.concat ([df,get_totals (driver, str(current_date))])
    driver.quit()
    
    df = df.reset_index(drop=True)
    df['calories'] = df['calories'].str.replace(',','').astype(int)
    
    return df

def graph_calories(df):
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    pd.options.plotting.backend = "plotly"
    fig = df['calories'].plot(kind='bar')
    return fig
    
@app.route("/", methods=["GET", "POST"])
def main():
    email = password = start_date = end_date = None
    if request.method == "POST":
        email = request.form.get("email")
        password = request.form.get("password")
        start_date = request.form.get("start_date")
        end_date = request.form.get("end_date")
        
        df = get_myfitnesspal_data(email,password,start_date,end_date)
        df = df.set_index('date')
        fig = graph_calories(df)
        
        return """
            <html>
            <head>
              <title>Remco's MyFitnessPal</title>
              <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
              {}
            </head>
            <body>
            <form action="/" method="get">
                <h1>Remco's MyFitnessPal Client Control</h1>
                <label>Email: {}</label>
                <br>
                <label>From {} to {}</label>
                <br>
                {}
                
                {}               
                
                <input type="submit" value="Go back to the form">
              </form>
            </body>
            </html>
        """.format(css_style_output,email, start_date, end_date, df.to_html(index=False, classes='data'),fig.to_html(full_html=False))

    return """
        <html>
        <head>
          <title>Remco's MyFitnessPal</title>
          <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
          {}
        </head>
        <body>
          <form action="/" method="post">
          <h1>Remco's MyFitnessPal Client Control</h1>
            <label for="email">Enter Your Email:</label>
            <input type="email" id="email" name="email" required>
            <br>

            <label for="password">Enter Your Password:</label>
            <input type="password" id="password" name="password" required>
            <br>

            <label for="start_date">Enter Start Date:</label>
            <input type="date" id="start_date" name="start_date" required>
            <br>

            <label for="end_date">Enter End Date:</label>
            <input type="date" id="end_date" name="end_date" required>
            <br>
  
            <input type="submit" value="Submit">
          </form>
        </body>
        </html>
    """.format(css_style)

if __name__ == "__main__":
    app.run()